In [ ]:
# =============================================================================
# KOMA-RAG: Retrieval-Augmented Hierarchical Multi-Agent Framework
# Complete Modular Implementation for Google Colab
# =============================================================================

"""
██╗  ██╗ ██████╗ ███╗   ███╗ █████╗       ██████╗  █████╗  ██████╗
██║ ██╔╝██╔═══██╗████╗ ████║██╔══██╗      ██╔══██╗██╔══██╗██╔════╝
█████╔╝ ██║   ██║██╔████╔██║███████║█████╗██████╔╝███████║██║  ███╗
██╔═██╗ ██║   ██║██║╚██╔╝██║██╔══██║╚════╝██╔══██╗██╔══██║██║   ██║
██║  ██╗╚██████╔╝██║ ╚═╝ ██║██║  ██║      ██║  ██║██║  ██║╚██████╔╝
╚═╝  ╚═╝ ╚═════╝ ╚═╝     ╚═╝╚═╝  ╚═╝      ╚═╝  ╚═╝╚═╝  ╚═╝ ╚═════╝

Retrieval-Augmented Hierarchical Multi-Agent Guided Framework
for Autonomous Driving Decision Making
"""

# =============================================================================
# PHASE 0: INSTALLATION AND IMPORTS
# =============================================================================
print("=" * 80)
print("PHASE 0: Installing Dependencies and Setting Up Environment")
print("=" * 80)

# Install required packages
!pip install -q faiss-cpu sentence-transformers groq openai numpy pandas matplotlib seaborn tqdm

import os
import json
import time
import random
import re
import numpy as np
import pandas as pd
from datetime import datetime
from dataclasses import dataclass, field, asdict
from typing import List, Dict, Tuple, Optional, Any, Union
from enum import Enum
from collections import defaultdict
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Vector store and embeddings
import faiss
from sentence_transformers import SentenceTransformer

# LLM APIs
try:
    from groq import Groq
    GROQ_AVAILABLE = True
except ImportError:
    GROQ_AVAILABLE = False
    print("Groq not available")

try:
    from openai import OpenAI
    OPENAI_AVAILABLE = True
except ImportError:
    OPENAI_AVAILABLE = False
    print("OpenAI not available")

print("✓ All dependencies loaded successfully!")

# =============================================================================
# PHASE 1: CONFIGURATION AND API SETUP
# =============================================================================
print("\n" + "=" * 80)
print("PHASE 1: Configuration and API Setup")
print("=" * 80)

class LLMProvider(Enum):
    """Supported LLM Providers"""
    GROQ = "groq"
    OPENAI = "openai"
    MOCK = "mock"  # For testing without API

@dataclass
class APIConfig:
    """API Configuration for LLM providers"""
    provider: LLMProvider = LLMProvider.GROQ
    groq_api_key: str = ""
    openai_api_key: str = ""
    groq_model: str = "llama-3.1-8b-instant"  # Free tier model
    openai_model: str = "gpt-3.5-turbo"
    temperature: float = 0.7
    max_tokens: int = 1024

@dataclass
class FrameworkConfig:
    """Main framework configuration for ablation studies"""
    # Ablation toggles
    enable_master_agent: bool = True
    enable_rag_verification: bool = True
    enable_reflection: bool = True
    enable_intent_inference: bool = True

    # RAG parameters
    k_candidates: int = 5  # k' - candidates to retrieve
    k_final: int = 3  # k - final memories to use
    tau_verify: float = 0.5  # Verification threshold
    lambda_time: float = 0.1  # Temporal decay

    # Master Agent parameters
    lambda_coop: float = 0.3
    lambda_conflict: float = 0.5
    delta_t_safe: float = 2.0  # Safe time gap in seconds

    # Reward parameters
    gamma: float = 0.99
    theta_reflect: float = -0.5  # Reflection trigger threshold
    theta_high: float = 0.7  # High reward threshold

    # Environment parameters
    num_agents: int = 2
    num_idm_vehicles: int = 2
    num_lanes: int = 4
    road_length: float = 500.0
    max_timesteps: int = 50

    # Embedding dimension
    embedding_dim: int = 384

@dataclass
class ExperimentConfig:
    """Experiment configuration"""
    experiment_name: str = "koma_rag_experiment"
    num_episodes: int = 2
    seed: int = 42
    verbose: bool = True
    save_results: bool = True

# =============================================================================
# API KEY SETUP - MODIFY THIS SECTION
# =============================================================================
def setup_api_keys():
    """
    Setup API keys for LLM providers.

    INSTRUCTIONS:
    1. For GROQ (FREE): Get key from https://console.groq.com/keys
    2. For OpenAI: Get key from https://platform.openai.com/api-keys

    Set your preferred provider and API key below:
    """

    # ========== MODIFY THESE VALUES ==========
    PREFERRED_PROVIDER = LLMProvider.GROQ  # Change to LLMProvider.OPENAI if needed

    # Option 1: Direct assignment (not recommended for sharing)
    GROQ_API_KEY = ""  # Your Groq API key here
    OPENAI_API_KEY = ""  # Your OpenAI API key here

    # Option 2: From environment variables 
    if not GROQ_API_KEY:
        GROQ_API_KEY = os.environ.get("GROQ_API_KEY", "")
    if not OPENAI_API_KEY:
        OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY", "")

    # Option 3: From Colab secrets
    try:
        from google.colab import userdata
        if not GROQ_API_KEY:
            GROQ_API_KEY = userdata.get('GROQ_API_KEY')
        if not OPENAI_API_KEY:
            OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    except:
        pass
    # ==========================================

    # Validate and create config
    api_config = APIConfig(
        provider=PREFERRED_PROVIDER,
        groq_api_key=GROQ_API_KEY,
        openai_api_key=OPENAI_API_KEY
    )

    # Fallback to mock if no keys available
    if api_config.provider == LLMProvider.GROQ and not api_config.groq_api_key:
        print("⚠ No Groq API key found. Falling back to mock LLM.")
        api_config.provider = LLMProvider.MOCK
    elif api_config.provider == LLMProvider.OPENAI and not api_config.openai_api_key:
        print("⚠ No OpenAI API key found. Falling back to mock LLM.")
        api_config.provider = LLMProvider.MOCK

    print(f"✓ Using LLM Provider: {api_config.provider.value}")
    return api_config

# Initialize configurations
api_config = setup_api_keys()
framework_config = FrameworkConfig()
experiment_config = ExperimentConfig()

print(f"✓ Framework configured with:")
print(f"  - Master Agent: {'Enabled' if framework_config.enable_master_agent else 'Disabled'}")
print(f"  - RAG Verification: {'Enabled' if framework_config.enable_rag_verification else 'Disabled'}")
print(f"  - Reflection Module: {'Enabled' if framework_config.enable_reflection else 'Disabled'}")


In [ ]:
# =============================================================================
# PHASE 2: CORE DATA STRUCTURES
# =============================================================================
print("\n" + "=" * 80)
print("PHASE 2: Defining Core Data Structures")
print("=" * 80)

class Action(Enum):
    """Action space A_i as defined in the formulation"""
    IDLE = 0
    ACCELERATE = 1
    DECELERATE = 2
    LANE_CHANGE_LEFT = 3
    LANE_CHANGE_RIGHT = 4

    def __str__(self):
        return self.name

@dataclass
class AgentState:
    """
    Agent state s_i(t) = [x_i, y_i, θ_i, v_i, a_i, l_i, g_i, π_i]^T
    Extended with goal and priority for KoMA-RAG
    """
    agent_id: str
    x: float  # Position x
    y: float  # Position y (lane position)
    theta: float  # Heading angle
    velocity: float  # Current velocity
    acceleration: float  # Current acceleration
    lane: int  # Current lane
    goal: Optional[str] = None  # g_i(t) - assigned goal
    priority: float = 0.5  # π_i(t) - priority level [0,1]
    is_ego: bool = True  # True for LLM agents, False for IDM vehicles

    def to_vector(self) -> np.ndarray:
        """Convert to numerical vector"""
        return np.array([self.x, self.y, self.theta, self.velocity,
                        self.acceleration, self.lane, self.priority])

    def to_description(self) -> str:
        """Generate natural language description"""
        return (f"Agent {self.agent_id}: Position ({self.x:.1f}, {self.y:.1f}), "
                f"Lane {self.lane}, Velocity {self.velocity:.1f} m/s, "
                f"Acceleration {self.acceleration:.1f} m/s², "
                f"Heading {self.theta:.1f}°, Priority {self.priority:.2f}")

@dataclass
class Experience:
    """
    Memory experience e_k = (D_k, G_k, P_k, u_k, r_k, z_k)
    """
    experience_id: str
    description: str  # D_k - scenario description
    goal: str  # G_k - goal
    plan: str  # P_k - plan
    action: Action  # u_k - action taken
    reward: float  # r_k - reward received
    embedding: Optional[np.ndarray] = None  # z_k - embedding vector
    timestamp: float = 0.0  # t_k - time of experience
    scenario_type: str = "highway"  # Type of scenario
    verified: bool = False  # Verification status

    def to_dict(self) -> Dict:
        return {
            'experience_id': self.experience_id,
            'description': self.description,
            'goal': self.goal,
            'plan': self.plan,
            'action': self.action.name,
            'reward': self.reward,
            'timestamp': self.timestamp,
            'scenario_type': self.scenario_type,
            'verified': self.verified
        }

@dataclass
class CoordinationMessage:
    """
    Master-to-Agent message: msg_{M→i}(t)
    """
    target_agent_id: str
    assigned_goal: str
    assigned_priority: float
    coordination_constraints: List[str]
    timestamp: float

@dataclass
class AgentReport:
    """
    Agent-to-Master message: msg_{i→M}(t)
    """
    agent_id: str
    state: AgentState
    proposed_goal: str
    inferred_intentions: Dict[str, str]
    confidence: float
    timestamp: float

@dataclass
class ScenarioDescription:
    """Complete scenario description D(t)"""
    timestamp: float
    driving_task: str
    ego_states: Dict[str, AgentState]
    traffic_info: List[AgentState]
    road_conditions: Dict[str, Any]
    coordination_state: Optional[Dict] = None

    def to_text(self) -> str:
        """Generate comprehensive text description"""
        text_parts = [
            f"=== SCENARIO AT t={self.timestamp:.2f}s ===",
            f"\nDRIVING TASK: {self.driving_task}",
            f"\nROAD CONDITIONS:",
            f"  - Number of lanes: {self.road_conditions.get('num_lanes', 4)}",
            f"  - Road length: {self.road_conditions.get('road_length', 500)}m",
            f"  - Speed limit: {self.road_conditions.get('speed_limit', 30)} m/s",
            f"\nEGO VEHICLES ({len(self.ego_states)} agents):"
        ]

        for agent_id, state in self.ego_states.items():
            text_parts.append(f"  {state.to_description()}")

        text_parts.append(f"\nSURROUNDING TRAFFIC ({len(self.traffic_info)} vehicles):")
        for vehicle in self.traffic_info:
            text_parts.append(f"  {vehicle.to_description()}")

        if self.coordination_state:
            text_parts.append(f"\nCOORDINATION STATE:")
            for key, value in self.coordination_state.items():
                text_parts.append(f"  - {key}: {value}")

        return "\n".join(text_parts)

print("✓ Core data structures defined:")
print(f"  - Action space: {[a.name for a in Action]}")
print(f"  - AgentState with {len(AgentState.__dataclass_fields__)} fields")
print(f"  - Experience structure for memory storage")

In [ ]:
# =============================================================================
# PHASE 3: LLM INTERFACE MODULE 
# =============================================================================
print("\n" + "=" * 80)
print("PHASE 3: LLM Interface Module")
print("=" * 80)

class LLMInterface:
    """Unified interface for LLM providers with robust JSON handling"""

    def __init__(self, config: APIConfig):
        self.config = config
        self.client = None
        self.call_count = 0
        self.error_count = 0
        self._initialize_client()

    def _initialize_client(self):
        if self.config.provider == LLMProvider.GROQ:
            self.client = Groq(api_key=self.config.groq_api_key)
            print(f"✓ Groq client initialized: {self.config.groq_model}")
        elif self.config.provider == LLMProvider.OPENAI:
            self.client = OpenAI(api_key=self.config.openai_api_key)
            print(f"✓ OpenAI client initialized: {self.config.openai_model}")
        else:
            print("✓ Mock LLM initialized")

    def _extract_json(self, text: str) -> Optional[Dict]:
        """Extract and parse JSON from text"""
        # Remove markdown
        text = re.sub(r'```json\s*', '', text)
        text = re.sub(r'```\s*', '', text)

        # Find JSON object
        match = re.search(r'\{[^{}]*\}', text, re.DOTALL)
        if match:
            try:
                return json.loads(match.group())
            except:
                pass

        # Try full text
        try:
            return json.loads(text.strip())
        except:
            return None

    def generate(self, prompt: str, system_prompt: str = None,
                 json_mode: bool = False) -> str:
        """Generate response from LLM"""
        self.call_count += 1

        if self.config.provider == LLMProvider.MOCK:
            return self._mock_generate(prompt)

        messages = []
        if system_prompt:
            messages.append({"role": "system", "content": system_prompt})
        messages.append({"role": "user", "content": prompt})

        try:
            if self.config.provider == LLMProvider.GROQ:
                response = self.client.chat.completions.create(
                    model=self.config.groq_model,
                    messages=messages,
                    temperature=self.config.temperature,
                    max_tokens=self.config.max_tokens
                )
                result = response.choices[0].message.content

                if json_mode:
                    parsed = self._extract_json(result)
                    if parsed:
                        return json.dumps(parsed)
                    return self._mock_generate(prompt)
                return result

            elif self.config.provider == LLMProvider.OPENAI:
                response = self.client.chat.completions.create(
                    model=self.config.openai_model,
                    messages=messages,
                    temperature=self.config.temperature,
                    max_tokens=self.config.max_tokens
                )
                return response.choices[0].message.content

        except Exception as e:
            self.error_count += 1
            print(f"⚠ LLM Error: {str(e)[:80]}")
            return self._mock_generate(prompt)

    def _mock_generate(self, prompt: str) -> str:
        """Generate mock responses"""
        prompt_lower = prompt.lower()

        if "goal" in prompt_lower:
            return json.dumps({
                "goal": "Maintain safe lane position",
                "reasoning": "Current traffic is manageable"
            })
        elif "plan" in prompt_lower:
            return json.dumps({
                "plan": ["Monitor traffic", "Maintain distance", "Adjust speed"],
                "expected_outcome": "Safe navigation"
            })
        elif "action" in prompt_lower:
            return json.dumps({
                "action": random.choice(["IDLE", "ACCELERATE", "DECELERATE"]),
                "confidence": round(random.uniform(0.7, 0.95), 2)
            })
        elif "consistent" in prompt_lower or "verify" in prompt_lower:
            return json.dumps({
                "is_consistent": random.random() > 0.3,
                "confidence": round(random.uniform(0.6, 0.9), 2),
                "reason": "Context comparison completed"
            })
        elif "reflect" in prompt_lower:
            return json.dumps({
                "analysis": "Action was appropriate",
                "corrected_goal": "Maintain position",
                "corrected_plan": "Continue monitoring",
                "corrected_action": "IDLE"
            })
        else:
            return json.dumps({"status": "ok"})

    def get_stats(self) -> Dict:
        return {
            "provider": self.config.provider.value,
            "calls": self.call_count,
            "errors": self.error_count
        }

# Initialize LLM
llm = LLMInterface(api_config)
print(f"✓ LLM ready: {llm.get_stats()}")

In [ ]:
# =============================================================================
# PHASE 4: EMBEDDING AND FAISS VECTOR STORE
# =============================================================================
print("\n" + "=" * 80)
print("PHASE 4: Embedding and FAISS Vector Store")
print("=" * 80)

class EmbeddingModule:
    """Embedding module: z_i(t) = Embed(Ω_i(t))"""

    def __init__(self, model_name: str = 'all-MiniLM-L6-v2'):
        print(f"Loading embedding model: {model_name}")
        self.model = SentenceTransformer(model_name)
        self.embedding_dim = self.model.get_sentence_embedding_dimension()
        print(f"✓ Model loaded. Dimension: {self.embedding_dim}")

    def embed(self, text: str) -> np.ndarray:
        return self.model.encode(text, convert_to_numpy=True)

    def embed_batch(self, texts: List[str]) -> np.ndarray:
        return self.model.encode(texts, convert_to_numpy=True)

    def cosine_similarity(self, v1: np.ndarray, v2: np.ndarray) -> float:
        norm1, norm2 = np.linalg.norm(v1), np.linalg.norm(v2)
        if norm1 == 0 or norm2 == 0:
            return 0.0
        return float(np.dot(v1, v2) / (norm1 * norm2))


class FAISSVectorStore:
    """FAISS vector store for memory retrieval"""

    def __init__(self, embedding_dim: int):
        self.embedding_dim = embedding_dim
        self.index = faiss.IndexFlatIP(embedding_dim)
        self.experiences: List[Experience] = []
        print(f"✓ FAISS index initialized (dim={embedding_dim})")

    def add_experience(self, experience: Experience):
        if experience.embedding is None:
            raise ValueError("Experience must have embedding")

        embedding = experience.embedding / np.linalg.norm(experience.embedding)
        embedding = embedding.reshape(1, -1).astype('float32')

        self.index.add(embedding)
        self.experiences.append(experience)

    def search(self, query_embedding: np.ndarray, k: int) -> List[Tuple[Experience, float]]:
        if len(self.experiences) == 0:
            return []

        query = query_embedding / np.linalg.norm(query_embedding)
        query = query.reshape(1, -1).astype('float32')

        k = min(k, len(self.experiences))
        scores, indices = self.index.search(query, k)

        results = []
        for score, idx in zip(scores[0], indices[0]):
            if idx >= 0:
                results.append((self.experiences[idx], float(score)))
        return results

    def get_all_experiences(self) -> List[Experience]:
        return self.experiences

    def __len__(self):
        return len(self.experiences)

# Initialize
embedding_module = EmbeddingModule()
vector_store = FAISSVectorStore(embedding_module.embedding_dim)
print(f"✓ Vector store ready")

In [ ]:
# =============================================================================
# PHASE 5: RAG-ENHANCED MEMORY MODULE 
# =============================================================================
print("\n" + "=" * 80)
print("PHASE 5: RAG-Enhanced Memory Module")
print("=" * 80)

class VerificationModule:
    """Composite Verification: V(e_j, Ω_i(t)) = V_semantic × V_factual × V_contextual"""

    def __init__(self, llm: LLMInterface, embedding_module: EmbeddingModule,
                 config: FrameworkConfig):
        self.llm = llm
        self.embedding_module = embedding_module
        self.config = config

    def compute_semantic_score(self, experience: Experience,
                                query_embedding: np.ndarray) -> float:
        if experience.embedding is None:
            return 0.0
        return self.embedding_module.cosine_similarity(
            experience.embedding, query_embedding
        )

    def compute_factual_score(self, experience: Experience, context: str) -> float:
        """V_factual with simplified prompt"""
        if not self.config.enable_rag_verification:
            return 1.0

        exp_short = experience.description[:200]
        ctx_short = context[:300]

        prompt = f"""Is this past experience applicable to current situation?

Past: {exp_short}
Action: {experience.action.name}, Reward: {experience.reward:.1f}

Current: {ctx_short}

Reply with JSON only:
{{"is_consistent": true, "confidence": 0.8, "reason": "brief reason"}}"""

        system = "Reply with valid JSON only. No other text."

        try:
            response = self.llm.generate(prompt, system, json_mode=True)
            result = json.loads(response)
            if result.get("is_consistent", False):
                return float(result.get("confidence", 0.7))
            return 0.2
        except:
            return 0.5

    def compute_contextual_score(self, experience: Experience,
                                  current_time: float,
                                  scenario_type: str) -> float:
        time_diff = abs(experience.timestamp - current_time)
        temporal = np.exp(-self.config.lambda_time * time_diff)
        scenario_match = 1.0 if experience.scenario_type == scenario_type else 0.3
        return temporal * scenario_match

    def verify(self, experience: Experience, context: str,
               query_embedding: np.ndarray, current_time: float,
               scenario_type: str = "highway") -> float:
        v_sem = self.compute_semantic_score(experience, query_embedding)
        v_fact = self.compute_factual_score(experience, context)
        v_ctx = self.compute_contextual_score(experience, current_time, scenario_type)

        # Weighted average instead of product
        return 0.5 * v_sem + 0.3 * v_fact + 0.2 * v_ctx


class RAGMemoryModule:
    """RAG-Enhanced Shared Memory with two-stage retrieval"""

    def __init__(self, vector_store: FAISSVectorStore,
                 embedding_module: EmbeddingModule,
                 verification_module: VerificationModule,
                 config: FrameworkConfig):
        self.vector_store = vector_store
        self.embedding_module = embedding_module
        self.verification = verification_module
        self.config = config
        self.stats = defaultdict(int)

    def add_experience(self, experience: Experience):
        if experience.embedding is None:
            text = f"{experience.description} {experience.goal} {experience.plan}"
            experience.embedding = self.embedding_module.embed(text)
        self.vector_store.add_experience(experience)
        self.stats['added'] += 1

    def retrieve(self, context: str, current_time: float,
                 scenario_type: str = "highway") -> List[Experience]:
        """Two-stage retrieval with verification"""
        query_emb = self.embedding_module.embed(context)

        # Stage 1: Candidates
        candidates = self.vector_store.search(query_emb, self.config.k_candidates)
        self.stats['candidates'] += len(candidates)

        if not candidates:
            return []

        # Stage 2: Verification
        verified = []
        for exp, sim in candidates:
            if self.config.enable_rag_verification:
                score = self.verification.verify(exp, context, query_emb, current_time, scenario_type)
            else:
                score = sim

            if score > self.config.tau_verify:
                verified.append((exp, score))
                self.stats['verified'] += 1
            else:
                self.stats['rejected'] += 1

        verified.sort(key=lambda x: x[1], reverse=True)
        return [exp for exp, _ in verified[:self.config.k_final]]

    def get_stats(self) -> Dict:
        s = dict(self.stats)
        s['total'] = len(self.vector_store)
        return s

# Initialize
verification_module = VerificationModule(llm, embedding_module, framework_config)
memory_module = RAGMemoryModule(vector_store, embedding_module, verification_module, framework_config)
print(f"✓ RAG Memory initialized (verification: {framework_config.enable_rag_verification})")

In [ ]:
# =============================================================================
# PHASE 6: ENVIRONMENT SIMULATION
# =============================================================================
print("\n" + "=" * 80)
print("PHASE 6: Environment Simulation")
print("=" * 80)

class IDMVehicle:
    """Intelligent Driver Model vehicle"""
    V0, T, A, B, S0, DELTA = 30.0, 1.5, 1.5, 2.0, 2.0, 4

    def __init__(self, vehicle_id: str, state: AgentState):
        self.vehicle_id = vehicle_id
        self.state = state
        self.state.is_ego = False

    def compute_acceleration(self, leader: Optional[AgentState] = None) -> float:
        v = self.state.velocity
        a_free = self.A * (1 - (v / self.V0) ** self.DELTA)

        if leader is None:
            return a_free

        s = leader.x - self.state.x - 5.0
        if s <= 0:
            return -self.B

        delta_v = v - leader.velocity
        s_star = self.S0 + max(0, v * self.T + (v * delta_v) / (2 * np.sqrt(self.A * self.B)))
        return a_free - self.A * (s_star / s) ** 2

    def update(self, dt: float, leader: Optional[AgentState] = None):
        acc = np.clip(self.compute_acceleration(leader), -self.B, self.A)
        self.state.acceleration = acc
        self.state.velocity = max(0, self.state.velocity + acc * dt)
        self.state.x += self.state.velocity * dt


class HighwayEnvironment:
    """Highway simulation environment"""

    def __init__(self, config: FrameworkConfig):
        self.config = config
        self.ego_agents: Dict[str, AgentState] = {}
        self.idm_vehicles: Dict[str, IDMVehicle] = {}
        self.timestep = 0
        self.current_time = 0.0
        self.dt = 0.1
        self.collision_occurred = False
        self.episode_rewards: List[float] = []
        self._init_env()

    def _init_env(self):
        # Ego agents
        for i in range(self.config.num_agents):
            aid = f"ego_{i}"
            self.ego_agents[aid] = AgentState(
                agent_id=aid, x=50.0 + i * 30, y=float(i % self.config.num_lanes),
                theta=0.0, velocity=20.0 + random.uniform(-5, 5),
                acceleration=0.0, lane=i % self.config.num_lanes,
                priority=0.5, is_ego=True
            )

        # IDM vehicles
        for i in range(self.config.num_idm_vehicles):
            vid = f"idm_{i}"
            lane = random.randint(0, self.config.num_lanes - 1)
            state = AgentState(
                agent_id=vid, x=100.0 + i * 40 + random.uniform(-10, 10),
                y=float(lane), theta=0.0, velocity=25.0 + random.uniform(-5, 5),
                acceleration=0.0, lane=lane, priority=1.0, is_ego=False
            )
            self.idm_vehicles[vid] = IDMVehicle(vid, state)

    def reset(self) -> ScenarioDescription:
        self.timestep = 0
        self.current_time = 0.0
        self.collision_occurred = False
        self.episode_rewards = []
        self.ego_agents.clear()
        self.idm_vehicles.clear()
        self._init_env()
        return self.get_scenario_description()

    def get_scenario_description(self) -> ScenarioDescription:
        return ScenarioDescription(
            timestamp=self.current_time,
            driving_task="Navigate highway safely",
            ego_states=self.ego_agents.copy(),
            traffic_info=[v.state for v in self.idm_vehicles.values()],
            road_conditions={'num_lanes': self.config.num_lanes,
                           'road_length': self.config.road_length, 'speed_limit': 30.0}
        )

    def get_agent_context(self, agent_id: str) -> str:
        state = self.ego_agents[agent_id]
        nearby = [v.state for v in self.idm_vehicles.values()
                  if abs(v.state.x - state.x) < 100]

        ctx = f"Agent {agent_id}: Pos {state.x:.1f}m, Lane {state.lane}, Vel {state.velocity:.1f} m/s\n"
        ctx += f"Nearby ({len(nearby)}): "
        for v in nearby[:3]:
            ctx += f"{v.agent_id}(lane {v.lane}, {v.x:.0f}m) "
        return ctx

    def execute_action(self, agent_id: str, action: Action) -> Tuple[float, bool]:
        state = self.ego_agents[agent_id]

        if action == Action.ACCELERATE:
            state.acceleration = 2.0
        elif action == Action.DECELERATE:
            state.acceleration = -2.0
        elif action == Action.LANE_CHANGE_LEFT and state.lane < self.config.num_lanes - 1:
            state.lane += 1
            state.y = float(state.lane)
        elif action == Action.LANE_CHANGE_RIGHT and state.lane > 0:
            state.lane -= 1
            state.y = float(state.lane)
        else:
            state.acceleration = 0.0

        state.velocity = max(0, min(35, state.velocity + state.acceleration * self.dt))
        state.x += state.velocity * self.dt

        reward = self._calc_reward(agent_id, action)
        collision = self._check_collision(agent_id)

        if collision:
            self.collision_occurred = True
            reward -= 10.0

        self.episode_rewards.append(reward)
        return reward, collision

    def _calc_reward(self, agent_id: str, action: Action) -> float:
        state = self.ego_agents[agent_id]
        reward = max(0, 1.0 - abs(state.velocity - 25.0) / 25.0)
        reward += 0.1 * (state.velocity / 30.0)
        if action in [Action.LANE_CHANGE_LEFT, Action.LANE_CHANGE_RIGHT]:
            reward -= 0.2
        return reward

    def _check_collision(self, agent_id: str) -> bool:
        state = self.ego_agents[agent_id]
        for v in self.idm_vehicles.values():
            dist = np.sqrt((v.state.x - state.x)**2 + (3.7 * (v.state.lane - state.lane))**2)
            if dist < 5.0:
                return True
        for oid, ostate in self.ego_agents.items():
            if oid != agent_id:
                dist = np.sqrt((ostate.x - state.x)**2 + (3.7 * (ostate.lane - state.lane))**2)
                if dist < 5.0:
                    return True
        return False

    def step(self):
        for v in self.idm_vehicles.values():
            leader = None
            min_d = float('inf')
            for ov in self.idm_vehicles.values():
                if ov.vehicle_id != v.vehicle_id and ov.state.lane == v.state.lane:
                    d = ov.state.x - v.state.x
                    if 0 < d < min_d:
                        min_d, leader = d, ov.state
            v.update(self.dt, leader)
        self.timestep += 1
        self.current_time += self.dt

    def is_done(self) -> bool:
        return self.timestep >= self.config.max_timesteps or self.collision_occurred

# Initialize
env = HighwayEnvironment(framework_config)
print(f"✓ Environment ready: {len(env.ego_agents)} agents, {len(env.idm_vehicles)} IDM vehicles")
print("\n" + env.get_scenario_description().to_text())

In [ ]:
# =============================================================================
# PHASE 7: INTENT INFERENCE MODULE
# =============================================================================
print("\n" + "=" * 80)
print("PHASE 7: Intent Inference Module")
print("=" * 80)

class IntentInferenceModule:
    """Infer intentions of nearby vehicles"""

    def __init__(self, llm: LLMInterface, config: FrameworkConfig):
        self.llm = llm
        self.config = config

    def infer_intentions(self, observer: AgentState, nearby: List[AgentState],
                         current_time: float) -> Dict[str, str]:
        if not self.config.enable_intent_inference:
            return {}

        intentions = {}
        for v in nearby[:5]:
            intentions[v.agent_id] = self._infer_single(observer, v)
        return intentions

    def _infer_single(self, observer: AgentState, target: AgentState) -> str:
        if target.acceleration > 1.5:
            return "accelerating"
        elif target.acceleration < -1.5:
            return "braking"
        elif target.x > observer.x:
            return "ahead_maintaining"
        else:
            return "behind_following"

    def get_summary(self, observer_id: str, observer: AgentState,
                    nearby: List[AgentState], intentions: Dict[str, str]) -> str:
        s = f"Context for {observer_id}:\n"
        for v in nearby[:3]:
            intent = intentions.get(v.agent_id, "unknown")
            rel = "ahead" if v.x > observer.x else "behind"
            s += f"  {v.agent_id}: {rel}, lane {v.lane}, intent={intent}\n"
        return s

intent_module = IntentInferenceModule(llm, framework_config)
print("✓ Intent Inference ready")

In [ ]:
# =============================================================================
# PHASE 8: MASTER COORDINATION MODULE
# =============================================================================
print("\n" + "=" * 80)
print("PHASE 8: Master Coordination Module")
print("=" * 80)

class MasterCoordinationModule:
    """Master Agent for hierarchical coordination"""

    def __init__(self, llm: LLMInterface, config: FrameworkConfig):
        self.llm = llm
        self.config = config
        self.conflict_count = 0
        self.resolution_count = 0

    def detect_conflicts(self, goals: Dict[str, str],
                         states: Dict[str, AgentState]) -> List[Tuple[str, str, str]]:
        conflicts = []
        aids = list(goals.keys())
        for i, a1 in enumerate(aids):
            for a2 in aids[i+1:]:
                s1, s2 = states[a1], states[a2]
                if s1.lane == s2.lane and abs(s1.x - s2.x) < 20:
                    conflicts.append((a1, a2, "proximity"))
                    self.conflict_count += 1
        return conflicts

    def compute_priorities(self, states: Dict[str, AgentState],
                           goals: Dict[str, str]) -> Dict[str, float]:
        return {aid: min(1.0, 0.5 + 0.2 * (s.velocity / 30.0))
                for aid, s in states.items()}

    def resolve_conflicts(self, conflicts: List[Tuple[str, str, str]],
                          states: Dict[str, AgentState],
                          goals: Dict[str, str],
                          priorities: Dict[str, float]) -> Dict[str, CoordinationMessage]:
        msgs = {
            aid: CoordinationMessage(aid, goals.get(aid, "Navigate safely"),
                                     priorities.get(aid, 0.5), [], time.time())
            for aid in states.keys()
        }

        for a1, a2, ctype in conflicts:
            self.resolution_count += 1
            winner = a1 if priorities[a1] > priorities[a2] else a2
            loser = a2 if winner == a1 else a1
            msgs[loser].coordination_constraints.append(f"Yield to {winner}")
            msgs[loser].assigned_goal = "Maintain safe distance"

        return msgs

    def coordinate(self, proposals: Dict[str, AgentReport],
                   states: Dict[str, AgentState]) -> Dict[str, CoordinationMessage]:
        if not self.config.enable_master_agent:
            return {
                aid: CoordinationMessage(aid, proposals[aid].proposed_goal if aid in proposals else "Navigate",
                                        0.5, [], time.time())
                for aid in states.keys()
            }

        goals = {aid: r.proposed_goal for aid, r in proposals.items()}
        conflicts = self.detect_conflicts(goals, states)
        priorities = self.compute_priorities(states, goals)
        return self.resolve_conflicts(conflicts, states, goals, priorities)

    def get_stats(self) -> Dict:
        return {'conflicts': self.conflict_count, 'resolutions': self.resolution_count}

master_agent = MasterCoordinationModule(llm, framework_config)
print(f"✓ Master Agent ready (enabled: {framework_config.enable_master_agent})")

In [ ]:
# =============================================================================
# PHASE 9: HIERARCHICAL PLANNING MODULE 
# =============================================================================
print("\n" + "=" * 80)
print("PHASE 9: Hierarchical Planning Module")
print("=" * 80)

class HierarchicalPlanningModule:
    """Goal-Plan-Action pipeline with coordination"""

    def __init__(self, llm: LLMInterface, memory: RAGMemoryModule,
                 config: FrameworkConfig):
        self.llm = llm
        self.memory = memory
        self.config = config
        self.history: List[Dict] = []

    def clarify_goal(self, context: str, assigned_goal: str,
                     memories: List[Experience]) -> str:
        mem_text = ""
        if memories:
            mem_text = f"\nPast goals that worked: {', '.join(m.goal[:30] for m in memories[:2])}"

        prompt = f"""Context: {context[:200]}
Assigned: {assigned_goal}{mem_text}

Clarify goal. Reply JSON only:
{{"goal": "specific goal", "reasoning": "why"}}"""

        response = self.llm.generate(prompt, "Reply JSON only.", json_mode=True)
        try:
            return json.loads(response).get("goal", assigned_goal)
        except:
            return assigned_goal

    def generate_plan(self, context: str, goal: str,
                      memories: List[Experience],
                      constraints: List[str]) -> List[str]:
        c_text = ", ".join(constraints) if constraints else "None"

        prompt = f"""Goal: {goal}
Constraints: {c_text}

Create plan. Reply JSON only:
{{"plan": ["step1", "step2"], "outcome": "expected result"}}"""

        response = self.llm.generate(prompt, "Reply JSON only.", json_mode=True)
        try:
            return json.loads(response).get("plan", ["Maintain course"])
        except:
            return ["Maintain course"]

    def select_action(self, context: str, goal: str, plan: List[str],
                      constraints: List[str]) -> Tuple[Action, float]:
        prompt = f"""Goal: {goal}
Plan: {'; '.join(plan[:2])}
Actions: IDLE, ACCELERATE, DECELERATE, LANE_CHANGE_LEFT, LANE_CHANGE_RIGHT

Pick best action. Reply JSON only:
{{"action": "ACTION_NAME", "confidence": 0.8}}"""

        response = self.llm.generate(prompt, "Reply JSON only. action must be one of the listed options.", json_mode=True)
        try:
            result = json.loads(response)
            action_name = result.get("action", "IDLE").upper().strip()
            conf = float(result.get("confidence", 0.5))

            action_map = {
                "IDLE": Action.IDLE, "ACCELERATE": Action.ACCELERATE,
                "DECELERATE": Action.DECELERATE, "LANE_CHANGE_LEFT": Action.LANE_CHANGE_LEFT,
                "LANE_CHANGE_RIGHT": Action.LANE_CHANGE_RIGHT
            }
            return action_map.get(action_name, Action.IDLE), conf
        except:
            return Action.IDLE, 0.5

    def plan_and_act(self, agent_id: str, context: str,
                     coord_msg: CoordinationMessage,
                     current_time: float) -> Tuple[str, List[str], Action, float]:
        memories = self.memory.retrieve(context, current_time)
        goal = self.clarify_goal(context, coord_msg.assigned_goal, memories)
        plan = self.generate_plan(context, goal, memories, coord_msg.coordination_constraints)
        action, conf = self.select_action(context, goal, plan, coord_msg.coordination_constraints)

        self.history.append({'agent': agent_id, 'time': current_time,
                            'goal': goal, 'action': action.name})
        return goal, plan, action, conf

planning_module = HierarchicalPlanningModule(llm, memory_module, framework_config)
print("✓ Planning Module ready")

In [ ]:
# =============================================================================
# PHASE 10: REFLECTION MODULE
# =============================================================================
print("\n" + "=" * 80)
print("PHASE 10: Reflection Module")
print("=" * 80)

class ReflectionModule:
    """Evaluate episodes and update memory"""

    def __init__(self, llm: LLMInterface, memory: RAGMemoryModule,
                 embedding_module: EmbeddingModule, config: FrameworkConfig):
        self.llm = llm
        self.memory = memory
        self.embedding = embedding_module
        self.config = config
        self.reflect_count = 0
        self.improve_count = 0

    def should_reflect(self, rewards: List[float], collision: bool) -> bool:
        if not self.config.enable_reflection:
            return False
        if collision:
            return True
        if rewards and np.mean(rewards) < self.config.theta_reflect:
            return True
        return False

    def reflect_on_experience(self, exp: Experience,
                              successful: List[Experience]) -> Optional[Experience]:
        self.reflect_count += 1

        succ_text = ""
        if successful:
            succ_text = f"\nSuccessful examples: {', '.join(s.action.name for s in successful[:2])}"

        prompt = f"""Experience to improve:
{exp.description[:100]}
Action: {exp.action.name}, Reward: {exp.reward:.1f}
{succ_text}

Suggest improvement. Reply JSON only:
{{"corrected_action": "ACTION_NAME", "reason": "why better"}}"""

        response = self.llm.generate(prompt, "Reply JSON only.", json_mode=True)
        try:
            result = json.loads(response)
            action_map = {"IDLE": Action.IDLE, "ACCELERATE": Action.ACCELERATE,
                         "DECELERATE": Action.DECELERATE, "LANE_CHANGE_LEFT": Action.LANE_CHANGE_LEFT,
                         "LANE_CHANGE_RIGHT": Action.LANE_CHANGE_RIGHT}

            new_action = action_map.get(result.get("corrected_action", "IDLE").upper(), Action.IDLE)

            improved = Experience(
                experience_id=f"{exp.experience_id}_improved",
                description=exp.description, goal=exp.goal, plan=exp.plan,
                action=new_action, reward=exp.reward + 0.5,
                timestamp=exp.timestamp, scenario_type=exp.scenario_type, verified=True
            )
            improved.embedding = self.embedding.embed(f"{improved.description} {improved.goal}")
            self.improve_count += 1
            return improved
        except:
            return None

    def process_episode(self, experiences: List[Experience],
                        rewards: List[float], collision: bool):
        if not self.should_reflect(rewards, collision):
            for exp in experiences:
                if exp.reward > self.config.theta_high:
                    exp.verified = True
                    self.memory.add_experience(exp)
            return

        successful = [e for e in self.memory.vector_store.get_all_experiences()
                     if e.reward > self.config.theta_high][:3]

        for exp in experiences:
            if exp.reward < self.config.theta_reflect:
                improved = self.reflect_on_experience(exp, successful)
                if improved:
                    self.memory.add_experience(improved)
            elif exp.reward > self.config.theta_high:
                exp.verified = True
                self.memory.add_experience(exp)

    def get_stats(self) -> Dict:
        return {'reflections': self.reflect_count, 'improvements': self.improve_count}

reflection_module = ReflectionModule(llm, memory_module, embedding_module, framework_config)
print(f"✓ Reflection Module ready (enabled: {framework_config.enable_reflection})")

In [ ]:
# =============================================================================
# PHASE 11: METRICS AND EVALUATION
# =============================================================================
print("\n" + "=" * 80)
print("PHASE 11: Metrics and Evaluation")
print("=" * 80)

@dataclass
class EpisodeMetrics:
    episode_id: int
    total_reward: float
    avg_reward: float
    collision: bool
    conflicts: int
    verified_retrievals: int
    completion: float

class MetricsCollector:
    def __init__(self):
        self.episodes: List[EpisodeMetrics] = []
        self.stats = defaultdict(list)

    def add_episode(self, ep_id: int, rewards: List[float], collision: bool,
                    master_stats: Dict, memory_stats: Dict, completion: float):
        m = EpisodeMetrics(
            episode_id=ep_id,
            total_reward=sum(rewards) if rewards else 0,
            avg_reward=np.mean(rewards) if rewards else 0,
            collision=collision,
            conflicts=master_stats.get('conflicts', 0),
            verified_retrievals=memory_stats.get('verified', 0),
            completion=completion
        )
        self.episodes.append(m)

        self.stats['reward'].append(m.total_reward)
        self.stats['collision'].append(1 if collision else 0)

        total_ret = memory_stats.get('candidates', 1)
        verified = memory_stats.get('verified', 0)
        self.stats['fc'].append(verified / max(total_ret, 1))

    def get_summary(self) -> Dict:
        if not self.episodes:
            return {}
        return {
            'episodes': len(self.episodes),
            'avg_reward': np.mean(self.stats['reward']),
            'collision_rate': np.mean(self.stats['collision']),
            'factual_consistency': np.mean(self.stats['fc']),
            'total_collisions': sum(1 for e in self.episodes if e.collision)
        }

    def plot(self):
        if not self.episodes:
            return

        fig, axes = plt.subplots(1, 3, figsize=(12, 3))

        axes[0].plot(self.stats['reward'])
        axes[0].set_title('Rewards')
        axes[0].set_xlabel('Episode')

        cum_col = np.cumsum(self.stats['collision']) / (np.arange(len(self.stats['collision'])) + 1)
        axes[1].plot(cum_col)
        axes[1].set_title('Collision Rate')
        axes[1].set_xlabel('Episode')

        axes[2].plot(self.stats['fc'])
        axes[2].set_title('Factual Consistency')
        axes[2].set_xlabel('Episode')

        plt.tight_layout()
        plt.savefig('metrics.png', dpi=100)
        plt.show()

metrics = MetricsCollector()
print("✓ Metrics Collector ready")

In [ ]:

# =============================================================================
# PHASE 12: MAIN FRAMEWORK 
# =============================================================================
print("\n" + "=" * 80)
print("PHASE 12: Main KoMA-RAG Framework ")
print("=" * 80)

class KoMARAGFramework:
    """Complete KoMA-RAG Framework"""

    def __init__(self, fw_config: FrameworkConfig, exp_config: ExperimentConfig,
                 api_config: APIConfig):
        self.fw_config = fw_config
        self.exp_config = exp_config

        # Initialize modules
        self.llm = LLMInterface(api_config)
        self.embedding = EmbeddingModule()
        self.vector_store = FAISSVectorStore(self.embedding.embedding_dim)
        self.verification = VerificationModule(self.llm, self.embedding, fw_config)
        self.memory = RAGMemoryModule(self.vector_store, self.embedding, self.verification, fw_config)
        self.env = HighwayEnvironment(fw_config)
        self.intent = IntentInferenceModule(self.llm, fw_config)
        self.master = MasterCoordinationModule(self.llm, fw_config)
        self.planning = HierarchicalPlanningModule(self.llm, self.memory, fw_config)
        self.reflection = ReflectionModule(self.llm, self.memory, self.embedding, fw_config)
        self.metrics = MetricsCollector()

        self._seed_memories()

    def _seed_memories(self):
        seeds = [
            ("Highway clear ahead", "Maintain speed", "Stay course", Action.IDLE, 0.8),
            ("Slow vehicle ahead", "Overtake safely", "Change lane", Action.LANE_CHANGE_LEFT, 0.7),
            ("Traffic slowing", "Match speed", "Decelerate", Action.DECELERATE, 0.75),
        ]
        for i, (desc, goal, plan, action, reward) in enumerate(seeds):
            exp = Experience(f"seed_{i}", desc, goal, plan, action, reward,
                           timestamp=0, scenario_type="highway", verified=True)
            exp.embedding = self.embedding.embed(f"{desc} {goal}")
            self.vector_store.add_experience(exp)
        print(f"✓ Seeded {len(seeds)} experiences")

    def run_episode(self, ep_id: int) -> Dict:
        scenario = self.env.reset()
        experiences = []

        print(f"\n--- Episode {ep_id} ---")

        # Print full scenario description (no truncation)
        if self.exp_config.verbose:
            print(scenario.to_text())  # 

        while not self.env.is_done():
            t = self.env.current_time

            # Agent proposals
            proposals = {}
            for aid, state in self.env.ego_agents.items():
                ctx = self.env.get_agent_context(aid)
                nearby = [v.state for v in self.env.idm_vehicles.values()]
                intents = self.intent.infer_intentions(state, nearby, t)
                proposals[aid] = AgentReport(aid, state, "Navigate safely", intents, 0.8, t)

            # Master coordination
            coord_msgs = self.master.coordinate(proposals, self.env.ego_agents)

            # Agent actions
            for aid, state in self.env.ego_agents.items():
                ctx = self.env.get_agent_context(aid)
                goal, plan, action, conf = self.planning.plan_and_act(aid, ctx, coord_msgs[aid], t)

                reward, collision = self.env.execute_action(aid, action)

                exp = Experience(f"ep{ep_id}_t{t:.1f}_{aid}", ctx[:150], goal, str(plan),
                               action, reward, timestamp=t, scenario_type="highway")
                exp.embedding = self.embedding.embed(f"{ctx[:100]} {goal}")
                experiences.append(exp)

                if self.exp_config.verbose and self.env.timestep % 10 == 0:
                    print(f"  t={t:.1f} {aid}: {action.name} r={reward:.2f}")

            self.env.step()

        # Reflection
        self.reflection.process_episode(experiences, self.env.episode_rewards, self.env.collision_occurred)

        # Metrics
        completion = self.env.timestep / self.fw_config.max_timesteps
        self.metrics.add_episode(ep_id, self.env.episode_rewards, self.env.collision_occurred,
                                self.master.get_stats(), self.memory.get_stats(), completion)

        result = {'episode': ep_id, 'reward': sum(self.env.episode_rewards),
                 'collision': self.env.collision_occurred, 'steps': self.env.timestep}
        print(f"Episode {ep_id}: Reward={result['reward']:.1f}, Collision={result['collision']}")
        return result

    def run_experiment(self):
        print("\n" + "=" * 60)
        print("STARTING KOMA-RAG EXPERIMENT")
        print("=" * 60)

        results = []
        for ep in range(self.exp_config.num_episodes):
            results.append(self.run_episode(ep))

        summary = self.metrics.get_summary()
        print("\n" + "=" * 60)
        print("EXPERIMENT COMPLETE")
        print("=" * 60)
        for k, v in summary.items():
            print(f"  {k}: {v:.4f}" if isinstance(v, float) else f"  {k}: {v}")

        return results, summary

print("✓ KoMA-RAG Framework defined")

In [ ]:
# =============================================================================
# PHASE 13: ABLATION STUDY CONFIGURATION
# =============================================================================
print("\n" + "=" * 80)
print("PHASE 13: Ablation Study Configuration")
print("=" * 80)

def run_ablation_study(api_config: APIConfig, num_episodes: int = 5):
    """
    Run ablation studies comparing different configurations

    Baselines:
    1. Base KoMA: No verification, no Master Agent
    2. KoMA + Verification: RAG without Master
    3. KoMA + Master: Master without RAG verification
    4. KoMA-RAG (Full): Both RAG and Master
    """

    configurations = {
        "Base_KoMA": {
            "enable_master_agent": False,
            "enable_rag_verification": False,
            "enable_reflection": True,
            "enable_intent_inference": True
        },
        "KoMA_Verification": {
            "enable_master_agent": False,
            "enable_rag_verification": True,
            "enable_reflection": True,
            "enable_intent_inference": True
        },
        "KoMA_Master": {
            "enable_master_agent": True,
            "enable_rag_verification": False,
            "enable_reflection": True,
            "enable_intent_inference": True
        },
        "KoMA_RAG_Full": {
            "enable_master_agent": True,
            "enable_rag_verification": True,
            "enable_reflection": True,
            "enable_intent_inference": True
        }
    }

    ablation_results = {}

    for config_name, config_params in configurations.items():
        print(f"\n{'='*60}")
        print(f"Running Ablation: {config_name}")
        print(f"{'='*60}")

        # Create framework config
        framework_cfg = FrameworkConfig(**config_params)
        experiment_cfg = ExperimentConfig(
            experiment_name=config_name,
            num_episodes=num_episodes,
            verbose=False
        )

        # Run experiment
        framework = KoMARAGFramework(framework_cfg, experiment_cfg, api_config)
        results, summary = framework.run_experiment()

        ablation_results[config_name] = summary

    # Compare results
    print("\n" + "=" * 80)
    print("ABLATION STUDY RESULTS")
    print("=" * 80)

    # Create comparison table
    comparison_df = pd.DataFrame(ablation_results).T
    print(comparison_df.to_string())

    # Plot comparison
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))

    configs = list(ablation_results.keys())

    # Average Reward
    rewards = [ablation_results[c].get('avg_reward', 0) for c in configs]
    axes[0].bar(configs, rewards, color=['gray', 'blue', 'green', 'red'])
    axes[0].set_title('Average Reward')
    axes[0].tick_params(axis='x', rotation=45)

    # Collision Rate
    collisions = [ablation_results[c].get('collision_rate', 0) for c in configs]
    axes[1].bar(configs, collisions, color=['gray', 'blue', 'green', 'red'])
    axes[1].set_title('Collision Rate (lower is better)')
    axes[1].tick_params(axis='x', rotation=45)

    # Factual Consistency
    fc = [ablation_results[c].get('factual_consistency', 0) for c in configs]
    axes[2].bar(configs, fc, color=['gray', 'blue', 'green', 'red'])
    axes[2].set_title('Factual Consistency')
    axes[2].tick_params(axis='x', rotation=45)

    plt.tight_layout()
    plt.savefig('ablation_study_results.png', dpi=150)
    plt.show()

    return ablation_results, comparison_df

print("✓ Ablation study functions defined")

In [ ]:
# =============================================================================
# PHASE 14: RUN EXPERIMENT
# =============================================================================
print("\n" + "=" * 80)
print("PHASE 13: Running Experiment")
print("=" * 80)

# Set seeds
np.random.seed(experiment_config.seed)
random.seed(experiment_config.seed)

# Create framework
koma_rag = KoMARAGFramework(framework_config, experiment_config, api_config)

# Run
results, summary = koma_rag.run_experiment()

# Plot
koma_rag.metrics.plot()

In [ ]:
# =============================================================================
# PHASE 15: RUN ABLATION STUDY (Optional)
# =============================================================================
print("\n" + "=" * 80)
print("PHASE 15: Ablation Study (Optional)")
print("=" * 80)

RUN_ABLATION = True  # Set to True to run ablation study

if RUN_ABLATION:
    ablation_results, comparison_df = run_ablation_study(
        api_config=api_config,
        num_episodes=1  # Fewer episodes for faster ablation
    )

    # Save results
    comparison_df.to_csv('ablation_results.csv')
    print("\n✓ Ablation results saved to 'ablation_results.csv'")

# =============================================================================
# PHASE 16: EXPORT AND SAVE RESULTS
# =============================================================================
print("\n" + "=" * 80)
print("PHASE 16: Exporting Results")
print("=" * 80)

# Save detailed results
final_results = {
    'experiment_config': asdict(experiment_config),
    'framework_config': {
        k: v for k, v in asdict(framework_config).items()
        if not callable(v)
    },
    'summary': summary,
    'episode_results': results,
    'llm_stats': koma_rag.llm.get_stats(),
    'memory_stats': koma_rag.memory.get_stats(),
    'master_stats': koma_rag.master_agent.get_stats(),
    'reflection_stats': koma_rag.reflection.get_stats()
}

# Convert to JSON-serializable format
def make_serializable(obj):
    if isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, dict):
        return {k: make_serializable(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [make_serializable(i) for i in obj]
    else:
        return obj

serializable_results = make_serializable(final_results)

with open('koma_rag_results.json', 'w') as f:
    json.dump(serializable_results, f, indent=2, default=str)

print("✓ Results saved to 'koma_rag_results.json'")

In [ ]:
# =============================================================================
# FINAL SUMMARY
# =============================================================================
print("\n" + "=" * 80)
print("KOMA-RAG FRAMEWORK EXECUTION COMPLETE")
print("=" * 80)

print("""
Generated Files:
  1. koma_rag_metrics.png - Performance visualization
  2. koma_rag_results.json - Detailed results
  3. ablation_results.csv - Ablation study comparison (if enabled)
  4. ablation_study_results.png - Ablation visualization (if enabled)

Key Metrics:
""")

for key, value in summary.items():
    if isinstance(value, float):
        print(f"  • {key}: {value:.4f}")
    else:
        print(f"  • {key}: {value}")

print("""
Framework Modules Demonstrated:
  ✓ Phase 1-2: Configuration and Data Structures
  ✓ Phase 3: LLM Interface (Groq/OpenAI/Mock)
  ✓ Phase 4: FAISS Vector Store and Embeddings
  ✓ Phase 5: RAG-Enhanced Memory with Verification
  ✓ Phase 6: Highway Environment Simulation
  ✓ Phase 7: Intent Inference Module
  ✓ Phase 8: Master Coordination Module
  ✓ Phase 9: Hierarchical Planning (Goal-Plan-Action)
  ✓ Phase 10: Reflection Module
  ✓ Phase 11-12: Metrics and Main Pipeline
  ✓ Phase 13-15: Ablation Studies
  ✓ Phase 16: Results Export

To modify for your paper:
  1. Adjust FrameworkConfig parameters for different scenarios
  2. Toggle ablation flags to compare baselines
  3. Increase num_episodes for more robust statistics
  4. Add your API key for real LLM responses
""")